# Plotting the OR2 ComCam Images
<br>Owner: **Leanne Guy** ([@leannep](https://github.com/LSSTDESC/DC2-analysis/issues/new?body=@leannep))
<br>Last Verified to Run: **2020-07-27** (by @leannp)
<br>Verified Stack Release: **v20.0.0** (need to change to w30)

This notbook dispalys and analyses the ComCam images from OR2

## Set Up

In [ ]:
# What version of the Stack am I using?
! echo $HOSTNAME
! eups list | grep lsst_distrib

### Set up 

In [ ]:
import os
import sys
import warnings

# Gen-2 Butler
from lsst.daf.persistence import Butler


# Filter some warnings printed bythe stack
# warnings.simplefilter("ignore", category=FutureWarning)
# warnings.simplefilter("ignore", category=UserWarning)

## OR2 raw images

Raw calibration images are stored by type; `bias`, `flat`, `dark` In these folders, there is a directory structure 
based on the date the images were acquired (or transferred/ingested?)

In [ ]:
raw_datadir = '/project/shared/comCam/CALIB_n1'
! ls $raw_datadir

In [ ]:
! ls $raw_datadir'/bias/2020-07-27/'

## OR2 reprocessed data products

In [ ]:
from astropy.io import fits
ORN1Datadir = '/project/shared/comCam/rerun/ops_rehearsal_n1'
biasRawFile = fits.open(ORN1Datadir + "/bias/2020-07-27/bias-R22-S10-det003_2020-07-27.fits")
#print(biasRawFile.info())
biasRawFileHdr = biasRawFile[0].header
#print(repr(biasRawFileHdr))
print(biasRawFileHdr['RAFTBAY'])
print(biasRawFileHdr['CCDSLOT'])
print(biasRawFileHdr['CALIB_ID'])
print(biasRawFileHdr['OBSTYPE'])
print(biasRawFileHdr['CCDSLOT'])
night1DataGroupId = "CALSET_20200727_2114"

In [ ]:
butler = Butler(ORN1Datadir)
dataType = biasRawFileHdr['OBSTYPE']
expId = '2020072700001'
dataId = {'expId': expId, 'detector': 8}
dataImage = butler.get(dataType, dataId=dataId)
print(str(dataImage))

In [ ]:
dataSetTypes = butler.getDatasetTypes()
print(len(dataSetTypes))

if "flat" in dataSetTypes:
  print("Flat is dataset type!")

# Look for a given datasetType in the full set of datasetTypes
list(dataSetTypes).index("dark")

In [ ]:
## Query the metadata
datasetType = 'raw' 
frmt = ['expId'] # str/tuple of keys to be returned. Options: 
dataId = {'imageType': 'FLAT'} # dict of partial dataId
butler.queryMetadata(datasetType, frmt, dataId=dataId)

In [ ]:
butler.getKeys('flat')

In [ ]:
## Use the buter to find out what dataIds are available for the various calibration dataset types
flats = butler.queryMetadata('raw', ['visit', 'detector', 'expId'], dataId={'imageType': 'FLAT'})
print((flats))
darks = butler.queryMetadata('raw', ['expId'], dataId={'imageType': 'DARK'})
print(len(darks))
biases = butler.queryMetadata('raw', ['expId'], dataId={'imageType': 'BIAS'})
print(len(biases))

In [ ]:
biases = butler.queryMetadata('bias', ['filter'], dataId={'expId': 2020072700023, 'detector': 1})
print(biases)

In [ ]:
## Use the buter to find out what dataIds are available for the 'bias' dataset type
biases = butler.queryMetadata('raw', ['detector'], dataId=dataId)
type(biases)

In [ ]:
# Plot images for a given exposure id: 2020072400020
import numpy as np
dataType = 'bias'
iCcd  = 3
expId = 2020072400020
dataId = {'expId': expId, 'detector': iCcd}
dataImage = butler.get(dataType, dataId=dataId)
imageArr = dataImage.maskedImage.image.array
varianceArr = dataImage.maskedImage.variance.array

vmin = np.percentile(arr[np.where(np.isfinite(arr))], 2)
vmax = np.percentile(arr[np.where(np.isfinite(arr))], 98)

In [ ]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.resources import INLINE

In [ ]:
output_notebook(resources=INLINE)

In [ ]:
from bokeh.charts import HeatMap


In [ ]:
p = figure(x_range=(0,4000), y_range=(0,4000))
p.image(image=[imageArr], x=0, y=0, dw=vmax, dh=vmax, palette="Viridis256", level="image")
show(p)